In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(0, './')
sys.path.insert(0, './toolbench')
sys.path.insert(0, './toolbench/inference')
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [ ]:
import argparse
import json
from toolbench.inference.Downstream_tasks.rapidapi import pipeline_runner
from toolbench.inference.Downstream_tasks.rapidapi import rapidapi_wrapper
from toolbench.inference.Algorithms.single_chain import single_chain

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--backbone_model', type=str, default="toolllama", required=False, help='chatgpt_function or davinci or toolllama')
parser.add_argument('--openai_key', type=str, default="", required=False, help='openai key for chatgpt_function or davinci model')
parser.add_argument('--model_path', type=str, default="your_model_path/", required=False, help='')
parser.add_argument('--tool_root_dir', type=str, default="your_tools_path/", required=True, help='')
parser.add_argument("--lora", action="store_true", help="Load lora model or not.")
parser.add_argument('--lora_path', type=str, default="your_lora_path if lora", required=False, help='')
parser.add_argument('--max_observation_length', type=int, default=1024, required=False, help='maximum observation length')
parser.add_argument('--observ_compress_method', type=str, default="truncate", choices=["truncate", "filter", "random"], required=False, help='observation compress method')
parser.add_argument('--method', type=str, default="CoT@1", required=False, help='method for answer generation: CoT@n,Reflexion@n,BFS,DFS,UCT_vote')
parser.add_argument('--input_query_file', type=str, default="", required=False, help='input path')
parser.add_argument('--output_answer_file', type=str, default="",required=False, help='output path')
parser.add_argument('--toolbench_key', type=str, default="",required=False, help='your toolbench key to request rapidapi service')
parser.add_argument('--rapidapi_key', type=str, default="",required=False, help='your rapidapi key to request rapidapi service')
parser.add_argument('--use_rapidapi_key', action="store_true", help="To use customized rapidapi service or not.")

args = parser.parse_args(['--model_path', 'toolbench/ToolLLaMA-2-7b/', 
                          '--input_query_file', 'data/toolformer_data_convertion/eval/G1_eval_query.json',
                          '--output_answer_file', 'data/answer/test_toollama_2',
                          '--toolbench_key', '2TLxGfLkzebcy7trDP4RzqcbnuAFypTHbKlRNvYCGuVsn5Svv3',
                          '--tool_root_dir', 'data/toolenv/tools'])

In [ ]:
runner = pipeline_runner(args)

In [ ]:
idx = 6
print(runner.task_list[idx][6])
print('#' * 100)
print(runner.task_list[idx][3])

In [ ]:
idx = 4
data_dict = runner.task_list[idx][3]
args = runner.task_list[idx][4]
tool_des = runner.task_list[idx][6]
env = rapidapi_wrapper(data_dict, tool_des, None, args, process_id=0)

In [ ]:
data_dict

In [ ]:
args

In [ ]:
env._step('address_monitor', '{}')

In [ ]:
runner.task_list[0][3]

Попробуем прогнать модель на G1_test

In [ ]:
process_id = 0
single_chain_max_step = 12

for k, task in enumerate(runner.task_list[1:]):
    method = task[0]
    backbone_model = task[1]
    query_id = task[2]
    data_dict = task[3]
    args = task[4]
    tool_des = task[6]
    env = rapidapi_wrapper(data_dict, tool_des, None, args, process_id=process_id)

    passat = int(method.split("@")[-1])
    chain = single_chain(llm=backbone_model, io_func=env, process_id=process_id)
    chain.start(pass_at=passat,
                single_chain_max_step=single_chain_max_step,
                answer=1)
    break

In [ ]:
chain.tree[0]

In [ ]:
chain.terminal_node[0].get_chain_result_from_this_node()

In [ ]:
env.tool_names

In [ ]:
env.task_description

In [ ]:
print(backbone_model.conversation_history[0]['content'])

In [ ]:
tool_des

Проблема в том, что почему-то в промпте нету описания тулов с параметрами

In [ ]:
with open('data/toolllama_G123_dfs_train.json', 'r') as f:
    data = json.load(f)
data[100]['conversations'][-1]['value']

Поменял промпт. Попробую теперь позапускать. Возможно, качество станет лучше

In [ ]:
process_id = 0
single_chain_max_step = 12

for k, task in enumerate(runner.task_list[70:]):
    method = task[0]
    backbone_model = task[1]
    query_id = task[2]
    data_dict = task[3]
    args = task[4]
    tool_des = task[6]
    env = rapidapi_wrapper(data_dict, tool_des, None, args, process_id=process_id)

    passat = int(method.split("@")[-1])
    chain = single_chain(llm=backbone_model, io_func=env, process_id=process_id)
    chain.start(pass_at=passat,
                single_chain_max_step=single_chain_max_step,
                answer=1)
    break

In [ ]:
test = [1, 2, 3]
str(test)

In [ ]:
backbone_model.conversation_history